# Week 3 Assignment - Toronto Neighbourhood Project
### Explore, segment, and cluster the neighborhoods in the city of Toronto based on the postalcode and borough information

## SECTION 1 - Creating pandas dataframe for Toronto through webscraping Wikipedia

Import libraries

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import requests
from bs4 import BeautifulSoup

Extraction information from Wikipedia and creating BeautifulSoup with parse HTML

In [2]:
#Extract url and convert to BeautifulSoup
#This URL is used as it contains postalcode, borough & neighborhood info, similar to the example shown on coursera
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto_data = requests.get(url).text
toronto_soup = BeautifulSoup(toronto_data, "html5lib")

#Separate out element <td> as this contains all relevant information
toronto_table = toronto_soup.find_all("td")

#View html to gauge layout of the information - comment out if required
toronto_table

[<td style="width:11%;">
 <p>M1A<br/><span style="font-size:85%;">Not assigned</span>
 </p>
 </td>,
 <td style="width:11%;">
 <p>M2A<br/><span style="font-size:85%;">Not assigned</span>
 </p>
 </td>,
 <td style="width:11%;">
 <p>M3A<br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>)</span>
 </p>
 </td>,
 <td style="width:11%;">
 <p>M4A<br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>)</span>
 </p>
 </td>,
 <td style="width:11%;">
 <p>M5A<br/><span style="font-size:85%;"><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a><br/>(<a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a> / <a href="/wiki/Harbourfront,_Toronto" title="Harbourfront, Toronto">Harbourfront</a>)</span>
 </p>
 </td>,
 <td style="width:11%;">
 <p>M6A

Extracting from BeautifulSoup and converting to pandas dataframe

Remarks: Based on the layout of the wikipedia page (<a href='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'>link</a>), for the purpose of this webscraping project, I will presume the following:
<ol>
    <li>The first 3 characters are the <u>postal codes</u></li>
    <li>Any names/labels <b>outside of brackets</b> are names of the <u>borough</u></li>
    <li>Any names/labels <b>within brackets</b> are names of the <u>neighborhood</u></li>
</ol>

In [3]:
df_toronto = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

for i in range(len(toronto_table)):
    try:
        Post=toronto_table[i].p.text[0:3]
        Bor_Neigh=toronto_table[i].span.text 
        if Bor_Neigh in "Not assigned":
            borough = "Not assigned"
            Neigh = "Not assigned"
        else:   
            borough = Bor_Neigh[0:Bor_Neigh.find('(')]
            Neigh = Bor_Neigh[Bor_Neigh.find('(')+1:Bor_Neigh.find(')')]
        df_toronto = df_toronto.append({"PostalCode":Post, "Borough":borough, "Neighborhood":Neigh}, ignore_index=True)
    except (AttributeError):
        pass
    
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


First clean up:
<ul>
    <li> M7R, M5W and M7Y have commercial related labelling - will need to separate them</li>
</ul>

In [4]:
#Check value of columns
print(df_toronto.loc[df_toronto['PostalCode'].isin(['M7R','M5W','M7Y'])])

    PostalCode                                            Borough  \
114        M7R   MississaugaCanada Post Gateway Processing Centre   
148        M5W     Downtown TorontoStn A PO Boxes25 The Esplanade   
168        M7Y  East TorontoBusiness reply mail Processing Cen...   

       Neighborhood  
114  Enclave of L4W  
148  Enclave of M5E  
168  Enclave of M4L  


In [5]:
# Clean up the 3 postal codes by adding values manually via pd.Series and append
cleanUpList = [pd.Series(['M7R','Mississauga','Mississauga'], index=df_toronto.columns),
              pd.Series(['M7R','Canada Post Gateway','Canada Post Gateway'], index=df_toronto.columns),
              pd.Series(['M7R','Processing Centre','Enclave of L4W'], index=df_toronto.columns),
              
              pd.Series(['M5W','Downtown Toronto','Downtown Toronto'], index=df_toronto.columns),
              pd.Series(['M5W','Stn A PO Boxes','Stn A PO Boxes'], index=df_toronto.columns),
              pd.Series(['M5W','25 The Esplanade','Enclave of M5E'], index=df_toronto.columns),
              
              pd.Series(['M7Y','East Toronto','East Toronto'], index=df_toronto.columns),
              pd.Series(['M7Y','Business reply mail','Business reply mail'], index=df_toronto.columns),
              pd.Series(['M7Y','Processing Centre','Processing Centre'], index=df_toronto.columns),
              pd.Series(['M7Y','969 Eastern','Enclave of M4L'], index=df_toronto.columns)]

temp = df_toronto.append(cleanUpList, ignore_index = True)
print(temp.loc[temp['PostalCode'].isin(['M7R','M5W','M7Y'])])

    PostalCode                                            Borough  \
114        M7R   MississaugaCanada Post Gateway Processing Centre   
148        M5W     Downtown TorontoStn A PO Boxes25 The Esplanade   
168        M7Y  East TorontoBusiness reply mail Processing Cen...   
180        M7R                                        Mississauga   
181        M7R                                Canada Post Gateway   
182        M7R                                  Processing Centre   
183        M5W                                   Downtown Toronto   
184        M5W                                     Stn A PO Boxes   
185        M5W                                   25 The Esplanade   
186        M7Y                                       East Toronto   
187        M7Y                                Business reply mail   
188        M7Y                                  Processing Centre   
189        M7Y                                        969 Eastern   

            Neighborhood  
114   

In [6]:
# Dropping the original M7R, M5W and M7Y row through index
df_toronto_2 = temp.drop([114,148,168])
print(df_toronto_2.loc[df_toronto_2['PostalCode'].isin(['M7R','M5W','M7Y'])])

    PostalCode              Borough         Neighborhood
180        M7R          Mississauga          Mississauga
181        M7R  Canada Post Gateway  Canada Post Gateway
182        M7R    Processing Centre       Enclave of L4W
183        M5W     Downtown Toronto     Downtown Toronto
184        M5W       Stn A PO Boxes       Stn A PO Boxes
185        M5W     25 The Esplanade       Enclave of M5E
186        M7Y         East Toronto         East Toronto
187        M7Y  Business reply mail  Business reply mail
188        M7Y    Processing Centre    Processing Centre
189        M7Y          969 Eastern       Enclave of M4L


Second clean up:
<ul>
    <li> Replacing '/' to ',' for borough with multiple neighborhood (as per project requirement)</li>
</ul>

In [7]:
df_toronto_2['Neighborhood'] = df_toronto_2['Neighborhood'].str.replace('/', ',')

#Drop those with "Not assigned"
df_toronto_3 = df_toronto_2[df_toronto_2.Borough != "Not assigned"]

# Final dataframe before adding latitude and longitude (drop=True to eliminate old index)
df_toronto_3.reset_index(drop=True, inplace=True)

## SECTION 2 - Adding latitude and longtitude information into the dataframe.

Geocoder is not working. Will use the data provided by coursera

In [8]:
#Read from local machine. URL provided by coursera threw SSLVeriCertificationError 
df_geo = pd.read_csv("/Users/dewipori/Documents/Python/IBMDataScience/CapstoneProject/GeoSpatial.csv")
#df_geo.head()
df_geo.shape

(103, 3)

In [9]:
#Join two dataframes
df_toronto_4 = df_toronto_3.join(df_geo.set_index('PostalCode'), on='PostalCode')
df_toronto_4.shape
df_toronto_4

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## SECTION 3 - Analysis of Toronto Neighborhood.

Looking into top venues in Toronto neighborhood, clustering and visualising on the world map

Import libraries

In [10]:
import json # library to handle JSON files
import folium # map rendering library
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans # import k-means from clustering stage
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Foursquare related information/details:

In [11]:
CLIENT_ID = 'S544KOFIE0XRIE4MJIAT3P5UGG0JSZDTQ2L4U31ULQDIXQQW' # your Foursquare ID
CLIENT_SECRET = 'B5Y2OLYWNZWYZUC1AGOUJGWMTVRSWPZ4V4KKC2HHHTQITBYK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S544KOFIE0XRIE4MJIAT3P5UGG0JSZDTQ2L4U31ULQDIXQQW
CLIENT_SECRET:B5Y2OLYWNZWYZUC1AGOUJGWMTVRSWPZ4V4KKC2HHHTQITBYK


### a) Exploring the Neighborhood

Retrieving information for top 100 venues within a 500m radius. Then storing information into pandas dataframe for further analysis

In [12]:
LIMIT = 100
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Call getNearbyVenues function (defined above)

In [13]:
toronto_venues = getNearbyVenues(names=df_toronto_4['Neighborhood'],
                                latitudes=df_toronto_4['Latitude'],
                                longitudes=df_toronto_4['Longitude'])

Checking how many venues return for each neighborhood

In [14]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood , Long Branch",9,9,9,9,9,9
"Bathurst Manor , Wilson Heights , Downsview North",23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4
"Bedford Park , Lawrence Manor East",24,24,24,24,24,24
Berczy Park,57,57,57,57,57,57
"Birch Cliff , Cliffside West",4,4,4,4,4,4
"Brockton , Parkdale Village , Exhibition Place",24,24,24,24,24,24
Business reply mail,15,15,15,15,15,15


### b) Analyse the neighborhood

Finding the most common venues by type (top 10)

In [15]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [16]:
#Re-ordering neighborhood to appear at front
temp_neigh = toronto_onehot['Neighborhood']
toronto_onehot = toronto_onehot.drop(columns=['Neighborhood'])
toronto_onehot.insert(loc=0, column='Neighborhood', value=temp_neigh)

toronto_onehot.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Function to sort the venues in descending order

In [17]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood.

In [55]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Lounge,Breakfast Spot,Clothing Store,Latin American Restaurant,Mexican Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
1,"Alderwood , Long Branch",Pizza Place,Pub,Pharmacy,Sandwich Place,Gym,Playground,Coffee Shop,Dance Studio,Molecular Gastronomy Restaurant,Modern European Restaurant
2,"Bathurst Manor , Wilson Heights , Downsview North",Bank,Coffee Shop,Restaurant,Middle Eastern Restaurant,Chinese Restaurant,Fried Chicken Joint,Sandwich Place,Diner,Mobile Phone Shop,Intersection
3,Bayview Village,Café,Chinese Restaurant,Bank,Japanese Restaurant,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel
4,"Bedford Park , Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Greek Restaurant,Butcher,Pub,Thai Restaurant,Liquor Store,Restaurant,Juice Bar


### c) Clustering and Visualising Toronto Neighborhood

Applying KMeans cluster for Toronto neighborhood and folium to display the locations

In [56]:
# set number of clusters
kclusters = 2
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

/var/folders/zy/3h4nt0m17m7198gz7cbf_c140000gn/T/ipykernel_8172/3672664181.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [57]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with df_toronto_4 (from earlier) to add latitude/longitude for each neighborhood
toronto_merged = df_toronto_4.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Fast Food Restaurant,Bus Stop,Park,Food & Drink Shop,Metro Station,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Pizza Place,Hockey Arena,French Restaurant,Coffee Shop,Portuguese Restaurant,Mediterranean Restaurant,Medical Center,Men's Store,Metro Station,Motel
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Theater,Café,Shoe Store,Sushi Restaurant,Beer Store
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Accessories Store,Women's Store,Arts & Crafts Store,Boutique,Vietnamese Restaurant,Furniture / Home Store,Event Space,Coffee Shop,Molecular Gastronomy Restaurant
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,1.0,Coffee Shop,Sushi Restaurant,Café,Yoga Studio,Persian Restaurant,Beer Bar,Spa,Smoothie Shop,Burrito Place,Sandwich Place


In [73]:
# There seems to be a bug where Cluster Labels is converted to FLOAT despite being INT after joining.
# 4 cases of NaN values were found, for ease of project, will drop these records and re-convert CLUSTER LABELS
toronto_merged.dropna(subset=['Cluster Labels'], inplace=True)
toronto_merged['Cluster Labels'].isnull().sum()
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.dtypes

PostalCode                 object
Borough                    object
Neighborhood               object
Latitude                  float64
Longitude                 float64
Cluster Labels              int64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

Visualise the clusters in the world map. We will start with Toronto, Ontario as the initial map position.

In [70]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.6534817, -79.3839347.


In [74]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

List of Venues that determine the cluster type

In [78]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Fast Food Restaurant,Bus Stop,Park,Food & Drink Shop,Metro Station,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant
10,North York,0,Pizza Place,Japanese Restaurant,Park,Bakery,Mexican Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station
11,Etobicoke,0,Bakery,Yoga Studio,Mexican Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station,Lounge
21,York,0,Park,Women's Store,Pool,Metro Station,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant
35,East YorkEast Toronto,0,Park,Convenience Store,Intersection,Yoga Studio,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
44,Scarborough,0,Bakery,Bus Line,Soccer Field,Intersection,Park,Ice Cream Shop,Bus Station,Miscellaneous Shop,Modern European Restaurant,Mobile Phone Shop
49,North York,0,Bakery,Construction & Landscaping,Park,Basketball Court,Trail,Yoga Studio,Mexican Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
52,North York,0,Park,Yoga Studio,Metro Station,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Men's Store
61,Central Toronto,0,Business Service,Park,Bus Line,Swim School,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Yoga Studio
66,North York,0,Park,Construction & Landscaping,Convenience Store,Yoga Studio,Mexican Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant


In [76]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1,Pizza Place,Hockey Arena,French Restaurant,Coffee Shop,Portuguese Restaurant,Mediterranean Restaurant,Medical Center,Men's Store,Metro Station,Motel
2,Downtown Toronto,1,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Theater,Café,Shoe Store,Sushi Restaurant,Beer Store
3,North York,1,Clothing Store,Accessories Store,Women's Store,Arts & Crafts Store,Boutique,Vietnamese Restaurant,Furniture / Home Store,Event Space,Coffee Shop,Molecular Gastronomy Restaurant
4,Queen's Park,1,Coffee Shop,Sushi Restaurant,Café,Yoga Studio,Persian Restaurant,Beer Bar,Spa,Smoothie Shop,Burrito Place,Sandwich Place
6,Scarborough,1,Fast Food Restaurant,Metro Station,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Men's Store,Hardware Store
7,North York,1,Gym,Café,Restaurant,Coffee Shop,Clothing Store,Supermarket,Chinese Restaurant,Sandwich Place,Beer Store,Japanese Restaurant
8,East York,1,Pizza Place,Athletics & Sports,Pharmacy,Flea Market,Bank,Gastropub,Intersection,Breakfast Spot,Pet Store,Gym / Fitness Center
9,Downtown Toronto,1,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Cosmetics Shop,Hotel,Japanese Restaurant,Fast Food Restaurant,Middle Eastern Restaurant,Ramen Restaurant
12,Scarborough,1,Bar,Yoga Studio,Mexican Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station,Lounge
13,North York,1,Gym,Café,Restaurant,Coffee Shop,Clothing Store,Supermarket,Chinese Restaurant,Sandwich Place,Beer Store,Japanese Restaurant
